In [ ]:
!pip install qdrant-client
!pip install sentence-transformers

In [ ]:
!mkdir ./bert-base-latin-uncased

!wget --no-check-certificate --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm=$(wget --quiet \
--save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
 'https://docs.google.com/uc?export=download&id=1Te_14UB-DZ8wYPhHGyDg7LadDTjNzpti' -O- | \
sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Te_14UB-DZ8wYPhHGyDg7LadDTjNzpti" \
-O ./latin_bert.tar

!rm -f /tmp/cookies.txt

!tar -xf ./latin_bert.tar --strip-components 1 -C ./bert-base-latin-uncased
!rm ./latin_bert.tar

In [17]:
from google.colab import files

import nltk
from nltk.tokenize import sent_tokenize

from qdrant_client import models, QdrantClient

from sentence_transformers import SentenceTransformer

In [ ]:
nltk.download('punkt')

In [ ]:
uploaded_file = files.upload()

In [6]:
sentences = sent_tokenize(
    str(
        uploaded_file[
            list(uploaded_file.keys())[0]
        ]
    )
)

In [ ]:
encoder = SentenceTransformer('./bert-base-latin-uncased/')

In [24]:
qdrant = QdrantClient(
    url='<url>',
    api_key='<token>',
)

In [ ]:
qdrant.recreate_collection(
    collection_name='base',
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

In [ ]:
qdrant.upload_records(
    collection_name='base',
    records=[
        models.Record(
            id=index,
            vector=encoder.encode(sentence).tolist(),
            payload={
                'name': str(list(uploaded_file.keys())[0]),
                'text': sentence
            }
        ) for index, sentence in enumerate(sentences)
    ]
)